In [1]:
import cv2
import os
import random
import numpy as np
from theia import utils
from dataset.generate import rotate_bound, imageOverlay

In [8]:
# Main sim dataset generation

overlay = cv2.imread('target_100.jpg')

with open("target_positions", "w+") as f:
    for img in os.listdir("raw_dataset"):
        background = cv2.imread("raw_dataset/"+img)
        height, width, channels = background.shape
        #overlay1 = noise(overlay)
        overlay2 = rotate_bound(overlay, random.randint(0,360))
        x = random.randint(200,width-200)
        y = random.randint(200,height-200)
        new = imageOverlay(background, overlay2, x=x, y=y, width=150)
        cv2.imwrite(f"sim_dataset/{img}",new)
        f.write(f"{img}, {x}, {y} \n")


In [ ]:
""" 
BLUR DETECTION 
https://www.pyimagesearch.com/2015/09/07/blur-detection-with-opencv/
"""
for img in os.listdir("sim_dataset"):
    image = cv2.imread("sim_dataset/"+img)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    fm = cv2.Laplacian(gray, cv2.CV_64F).var()
    text = "Not Blurry"
    # if the focus measure is less than the supplied threshold,
    # then the image should be considered "blurry"
    if fm < 100:
        text = "Blurry"
    # show the image
    cv2.putText(image, "{}: {:.2f}".format(text, fm), (100, 100),
        cv2.FONT_HERSHEY_SIMPLEX, 5, (0, 0, 255), 3)
    utils.display(image)

In [2]:

import cv2
import numpy as np
from theia import utils
import os

image_number = 50
files = [f for f in os.listdir('./dataset/sim_dataset/')]
image = cv2.imread('./dataset/sim_dataset/' + files[image_number])
print(files[image_number])

pts1 = np.array([
    [0,0],
    [4608, 0],
    [0, 3456],
    [4608, 3456]
])

def warp(x_warp, y_warp, image):
    pts2 = np.array([
        [x_warp,y_warp],
        [4608-x_warp, 0],
        [0, 3456-y_warp],
        [4608, 3456]
    ])

    h, mask = cv2.findHomography(pts1, pts2, cv2.RANSAC, 5.0)

    im1Reg = cv2.warpPerspective(image, h, (4608, 3456))

    return im1Reg, h

#utils.display(warp(1000,1000, image)[0])

h = warp(900,900, image)[1]
centre = (2309, 3193)

def warp_coord(centre, h):
    """ https://docs.opencv.org/4.x/da/d54/group__imgproc__transform.html#gaf73673a7e8e18ec6963e3774e6a94b87 """
    return (
        int((h[0][0]*centre[0] + h[0][1]*centre[1] + h[0][2])/(h[2][0]*centre[0] + h[2][1]*centre[1] + h[2][2])),
        int((h[1][0]*centre[0] + h[1][1]*centre[1] + h[1][2])/(h[2][0]*centre[0] + h[2][1]*centre[1] + h[2][2]))
    )
warp_coord(centre, h)

IMG_160729_071816_0092_RGB.JPG


In [1]:
# Single threaded version of transformed dataset generation

ref = utils.read_positions("./dataset/target_positions")

image_number = 50
files = [f for f in os.listdir('./dataset/sim_dataset/')]
#image = cv2.imread('./dataset/sim_dataset/' + files[image_number])
#print(files[image_number])


for file in files:
    image = cv2.imread('./dataset/sim_dataset/' + file)
    print(file)
    with open("./dataset/target_positions_transformed", "w+") as f:
        for i in range(0,1000,100):
            for j in range(0,1000,100):
                transform, h = warp(i, j, image)
                name = f"{file}_{i}-{j}.jpg"
                cv2.imwrite(f"./dataset/transform_dataset/{name}", transform)
                centre = ref[file]
                x, y = warp_coord(centre, h)
                #marked = cv2.circle(transform, (x,y), 10, (0,0,255), thickness=-1)
                #utils.display(marked)
                f.write(f"{name}, {x}, {y} \n")

    

In [1]:
from multiprocessing import Pool
from theia import utils
from dataset.generate import create_transform_images
ref = utils.read_positions("./dataset/target_positions")

image_number = 50
files = [f for f in os.listdir('./dataset/sim_dataset/')]
#image = cv2.imread('./dataset/sim_dataset/' + files[image_number])
#print(files[image_number])

p = Pool(processes=12)
#result = p.map(create_transform_images, files)
result = p.map(create_transform_images, files)


In [6]:
with open("./dataset/target_positions_transformed", "w+") as f:
    r = [item for sublist in result for item in sublist]
    f.writelines(r)